# Reporter Agent

This notebook illustrates how the very first version of this agent was developed


In [3]:
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

from langgraph.graph import START, END, StateGraph
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Load environment variables
env_path = project_root / '.env'
load_dotenv(dotenv_path=env_path)

print("✓ Imports successful")
print(f"✓ Loading .env from: {env_path}")

✓ Imports successful
✓ Loading .env from: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.env


In [4]:
slug = "socotra_inside_the_alien_island"

## Prompts

In [5]:
# ============= FROZEN PROMPTS =============

magazine_profile = """ Agentic Newsroom is a magazine of discovery and wonder.
It explores the hidden corners of the planet and the universe, cutting-edge scientific mysteries, strange natural phenomena, ancient civilization, unusual places, and the latest in space exploration.
Our editorial voice blends scientific rigor, vivid storytelling, a touch of wonder, and strong narrative.
Articles are deeply researched, visually descriptive, and rich in detail.
We aim to make complex ideas accessible, without dumbing them down and to evoke awe, curiosity, and sense of scale.
"""

magazine_guardrails = """
Agentic Newsroom adheres to the following editorial standards in all published writing:

1. Agentic Newsroom does not produce or publish any form of hate speech, harassment, racism, sexism, or discriminatory content.
2. Articles may occasionally discuss pseudoscience, fringe theories, myths, or unverified claims when relevant to cultural or historical context. 
   However, Agentic Newsroom does not endorse such claims and always presents them with clear scientific, historical, or factual framing.
3. All information must be presented accurately, responsibly, and with respect for both scientific consensus and ethical journalism.
4. When uncertainty exists, Agentic Newsroom clearly distinguishes between established fact, informed interpretation, and speculation.
"""

editor_in_chief_profile = """
Your name is Margaret. You are the Editor in Chief of Agentic Newsroom.
You have more than thirty years of experience leading major science and geography magazines.
Your role is to make the final publication decision on every article.
You are the last line of editorial judgment, ensuring that every published piece meets Agentic Newsroom's standards for accuracy, clarity, narrative quality, and editorial integrity.
You balance editorial vision with practical publishing considerations.
You are decisive but fair, and your feedback is always constructive and specific.
"""

editor_in_chief_prompt = """You are the Editor in Chief of The Agentic Newsroom, responsible for making the final approval decision on articles.

<Magazine Profile>
{magazine_profile}
</Magazine Profile>

<Magazine Guardrails>
Strictly adhere to the following editorial standards:
{magazine_guardrails}
</Magazine Guardrails>

<Editor in Chief Profile>
{editor_in_chief_profile}
</Editor in Chief Profile>

<Task>
You will receive:
1. The original Story Brief that was assigned to the reporter
2. The Final Article that has been edited and polished

Your task is to review the final article against the story brief and make a publication decision.
</Task>

<Instructions>
Review the article carefully and evaluate:
1. **Alignment with Brief**: Does the article deliver on the topic, angle, and key questions from the story brief?
2. **Editorial Quality**: Is the writing clear, engaging, and well-structured?
3. **Factual Integrity**: Are claims properly attributed? Does it meet our guardrails?
4. **Narrative Strength**: Does it evoke curiosity and wonder while maintaining rigor?
5. **Length**: Is it within reasonable range of the target length?

Make one of two decisions:
- **approve**: The article is ready for publication
- **reject**: The article needs significant revision (provide specific reasons)

If you approve, you may include optional editorial notes.
If you reject, you must provide clear, actionable revision instructions.
</Instructions>

<Story Brief>
{story_brief}
</Story Brief>
"""

print("✓ Editor in Chief prompt defined")

✓ Editor in Chief prompt defined


## States and Structured Outputs

In [6]:
# ============= FROZEN SCHEMAS =============

from typing import List, Optional, TypedDict
from pydantic import BaseModel, Field

class StoryBrief(BaseModel):
    """Story brief created by the Assignment Editor."""
    topic: str = Field(..., description="Clear statement of what the story is about")
    angle: str = Field(..., description="The specific approach or perspective to take")
    length: int = Field(..., description="The length of the article in words")
    key_questions: List[str] = Field(..., description="3-5 questions the article should answer")

class FinalArticle(BaseModel):
    final_article: str = Field(..., description="The final article text")

class EditorDecision(BaseModel):
    """Editorial decision from the Editor-in-Chief."""
    decision: str = Field(..., description="Editorial decision: 'approve' or 'reject'")
    editor_notes: List[str] = Field(default_factory=list, description="General notes if approved")
    rejection_reasons: List[str] = Field(default_factory=list, description="Specific reasons if decision is 'reject'")

class EditorInChiefState(TypedDict):
    """State for the Editor in Chief subgraph."""
    story_brief: StoryBrief
    final_article: FinalArticle
    editor_decision: Optional[EditorDecision]

## Tools and Configuration

In [7]:
model = init_chat_model(model="openai:gpt-5.1", reasoning_effort="medium")
# model = init_chat_model(model="openai:gpt-5-mini", reasoning_effort="minimal")

print("✓ Configuration complete")

✓ Configuration complete


## Nodes

In [8]:
def review_article(state: EditorInChiefState):
    """
    Editor in Chief reviews the final article against the story brief
    and makes the approval/rejection decision.
    """
    story_brief = state["story_brief"]
    final_article = state["final_article"]
    
    # Format story brief for the prompt
    brief_str = story_brief.model_dump_json(indent=2)
    
    # Build the system message (without final article)
    system_msg_content = editor_in_chief_prompt.format(
        magazine_profile=magazine_profile,
        magazine_guardrails=magazine_guardrails,
        editor_in_chief_profile=editor_in_chief_profile,
        story_brief=brief_str
    )
    
    # Pass final article as HumanMessage
    messages = [
        SystemMessage(content=system_msg_content),
        HumanMessage(content=f"Here is the final article for your review:\n\n{final_article.final_article}")
    ]
    
    structured_model = model.with_structured_output(EditorDecision)
    response = structured_model.invoke(messages)
    
    return {
        "editor_decision": response
    }

print("✓ Node defined")

✓ Node defined


## Graph

In [9]:
builder = StateGraph(EditorInChiefState)
builder.add_node("review_article", review_article)
builder.add_edge(START, "review_article")
builder.add_edge("review_article", END)

editor_in_chief_agent = builder.compile()

print("✓ Graph compiled")
print("\nGraph structure (ASCII):")
print(editor_in_chief_agent.get_graph().draw_ascii())

✓ Graph compiled

Graph structure (ASCII):
  +-----------+    
  | __start__ |    
  +-----------+    
         *         
         *         
         *         
+----------------+ 
| review_article | 
+----------------+ 
         *         
         *         
         *         
    +---------+    
    | __end__ |    
    +---------+    


## Test Run

In [10]:
def load_story_brief(slug: str) -> StoryBrief:
    """Load a StoryBrief from the tmp directory."""
    path = project_root / "tmp" / slug / "story_brief.json"
    if not path.exists():
        raise FileNotFoundError(f"StoryBrief not found at {path}")
    
    with open(path, "r") as f:
        data = json.load(f)
    return StoryBrief(**data)

def load_final_article(slug: str) -> FinalArticle:
    """Load a FinalArticle from the tmp directory."""
    path = project_root / "tmp" / slug / "final_article.json"
    if not path.exists():
        raise FileNotFoundError(f"FinalArticle not found at {path}")
    
    with open(path, "r") as f:
        data = json.load(f)
    return FinalArticle(**data)

# Load inputs
story_brief = load_story_brief(slug)
final_article = load_final_article(slug)

print("✓ Story Brief Loaded")
print(f"  Topic: {story_brief.topic}")
print(f"  Target Length: {story_brief.length} words")
print(f"\n✓ Final Article Loaded")
print(f"  Article Length: {len(final_article.final_article)} characters")

✓ Story Brief Loaded
  Topic: Socotra: Inside the ‘Alien’ Island Where Evolution Went Off-Script
  Target Length: 900 words

✓ Final Article Loaded
  Article Length: 8261 characters


In [11]:
result = editor_in_chief_agent.invoke({
    "story_brief": story_brief,
    "final_article": final_article
})

print("✓ Agent execution complete")

/Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.venv/lib/python3.12/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


✓ Agent execution complete


## Display Editor Decision

In [12]:
from IPython.display import display, Markdown

def format_editor_decision(decision: EditorDecision) -> str:
    """Format an EditorDecision as markdown."""
    md = []
    
    # Header
    md.append("# Editor in Chief Decision")
    md.append("")
    md.append("---")
    md.append("")
    
    # Decision
    md.append(f"### Decision: **{decision.decision.upper()}**")
    md.append("")
    
    # Editor Notes (if approved)
    if decision.editor_notes:
        md.append("### Editor Notes")
        md.append("")
        for i, note in enumerate(decision.editor_notes, 1):
            md.append(f"{i}. {note}")
        md.append("")
    
    # Rejection Reasons (if rejected)
    if decision.rejection_reasons:
        md.append("### Rejection Reasons")
        md.append("")
        for i, reason in enumerate(decision.rejection_reasons, 1):
            md.append(f"{i}. {reason}")
        md.append("")
    
    return "\n".join(md)

# Display the decision
if "editor_decision" in result and result["editor_decision"]:
    markdown_output = format_editor_decision(result["editor_decision"])
    display(Markdown(markdown_output))
else:
    print("⚠️ No editor decision found in output.")

# Editor in Chief Decision

---

### Decision: **REJECT**

### Rejection Reasons

1. The piece is overly centered on the dragon’s blood tree and its monitoring studies, leaving much of Socotra’s wider “alien” biodiversity underdeveloped. The brief explicitly calls for explaining what makes Socotra’s landscape and organisms so different—from desert roses to cave-dwelling species—yet readers only get a passing sense of flora beyond *Dracaena cinnabari* and none of the island’s striking animals or marine life. Please add at least one compact section that vividly sketches a few other emblematic endemics (e.g., Socotra desert rose, cucumber tree, frankincense, distinctive reptiles/birds, freshwater cave fauna, or bioluminescent beaches), tying them back to the island’s isolation and climate.
2. The lived reality of Socotra’s people and their relationship with the environment is not sufficiently addressed. The brief asks how local cultures and traditional practices have shaped, protected, or strained ecosystems. At present, we get language, archaeology, and political history, but almost nothing on daily life or land-use: pastoralism and goat herding, harvesting of resins (including dragon’s blood), traditional water management, sacred groves or informal protection regimes, and how these practices have historically limited or exacerbated pressure on habitats. Please add reporting or sourced context that shows Socotrans as active stewards and users of the landscape, not just inhabitants in a political timeline.
3. Modern pressures—conflict, conservation challenges, and tourism—are treated too generically relative to the brief’s central tension: can Socotra remain otherworldly in a rapidly changing world? The political chronology is solid, but its ecological and social consequences are barely sketched. Likewise, the tourism paragraph hedges with “data are scarce” and then retreats into abstractions. Please:
- Connect Yemen’s instability and changing control on the island to concrete on-the-ground effects (e.g., governance gaps, conservation enforcement, illegal plant/animal collection, infrastructure projects, or disruptions from recent cyclones).
- Give at least one specific, sourced example or figure that illustrates the rise of expedition/Instagram tourism and its visible impacts (4x4 tracks on plateaus, campsite waste, pressure on key sites like Firmihin or Hoq Cave, or, conversely, community-based tourism efforts).
4. The article frames Socotra as a “natural laboratory,” but the scientific narrative remains narrowly focused on dragon’s blood tree surveys and habitat modeling. The brief also asks about Socotra’s broader role in understanding evolution and climate resilience. Consider adding a short, integrative paragraph that highlights other lines of research—such as studies of arid-adapted plants as models for drought resilience, cave or freshwater fauna evolution in isolation, or genetic work on the Modern South Arabian language group and human migration—so the reader feels the full scope of Socotra as a multi-disciplinary scientific frontier.
5. Narratively, the opening is strong but the sense of wonder thins out as the piece progresses and becomes citation-heavy around a single journal (*Forests*). Our house style favors rigor without letting the prose feel like a literature review. In revisions, please:
- Smooth or consolidate the repeated references to the same *Forests* and PMC papers so they support the narrative rather than dominate it.
- Reinforce sensory description and a human presence in at least one of the later sections (for example, a goatherd passing under dragon’s blood trees, resin sellers in Hadibo, or guides leading visitors into Hoq Cave) and return more explicitly in the conclusion to the central tension of whether this “alien” ecosystem can persist under 21st-century pressures. This can be done in 1–2 additional paragraphs without significantly increasing length.
6. Length is broadly acceptable, but the added material requested above will likely push it over the target. To keep it within a reasonable range (~900–1,100 words), trim some of the denser methodological detail (e.g., exact plot numbers and height thresholds in the Firmihin inventory) and streamline the mid-section where multiple similar dragon’s blood studies are cited. Preserve key findings (endemic percentage, approximate tree population, sensitivity to fog belts) while freeing space for the broader ecological and cultural context the brief requires.


In [13]:
def save_editor_decision(decision: EditorDecision, slug: str):
    """
    Save an EditorDecision to the slug directory.
    
    Saves both:
    - editor_decision.json (can be loaded back into EditorDecision object)
    - editor_decision.md (human-readable markdown version)
    
    Args:
        decision: The EditorDecision to save
        slug: The article slug (directory name in tmp/)
    
    Returns:
        Path to the article directory
    """
    # Define paths
    tmp_dir = project_root / "tmp"
    article_dir = tmp_dir / slug
    
    # Ensure directory exists
    article_dir.mkdir(parents=True, exist_ok=True)
    
    # Save JSON (can be loaded back into EditorDecision)
    json_path = article_dir / "editor_decision.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        f.write(decision.model_dump_json(indent=2))
    
    # Save Markdown version using the formatter
    md_path = article_dir / "editor_decision.md"
    markdown_content = format_editor_decision(decision)
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    print(f"Editor decision saved to: {article_dir}")
    print(f"   - JSON: {json_path.name}")
    print(f"   - Markdown: {md_path.name}")
    
    return article_dir

# Save the editor decision
if "editor_decision" in result and result["editor_decision"]:
    saved_dir = save_editor_decision(result["editor_decision"], slug)
else:
    print("No editor decision to save")

Editor decision saved to: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/tmp/socotra_inside_the_alien_island
   - JSON: editor_decision.json
   - Markdown: editor_decision.md
